# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [16]:
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import pickle
import pandas as pd
pd.set_option('display.max_columns', 200)

from card import Card
from board import Board
from player import Player
from rule import Rule
import optimal_strategy as opt

import multiprocessing
import time
from multiprocessing.pool import ThreadPool
import threading
from tqdm import notebook
notebook.tqdm.pandas()

from euchre_lib import *
from search_lib import *

In [3]:
notebook.tqdm.get_lock().locks = []

# Create dataset

In [5]:
mx_thresh = 75; N_CPUS = 8
thresholds = [mx_thresh]*N_CPUS
n_hands = int(1e4)
n_epochs = 400
# get 32 million = 32e6 = 8 CPUS x 1e4 hands x 4e2 epochs

In [ ]:
#parallel_search_wrapper(thresholds, MAX_CPUS=N_CPUS, n_epochs=n_epochs, n_hands=n_hands, opp_thresh=mx_thresh, title='Making dataset')

In [ ]:
#def parallel_search_wrapper(thresholds, MAX_CPUS=8, n_epochs=10, n_hands=100, opp_thresh=None,
#                            title=None, ROOT_DIR=os.getcwd()+'/'):

# Predict Expert Moves

In [6]:
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

2.4.0


In [9]:
infolder = os.path.join(os.getcwd(), 'thresholds', 'thresh70')
data = read_all_hands(infolder)
data.shape

  0%|          | 0/100 [00:00<?, ?it/s]

(1000000, 122)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Columns: 122 entries, p0c1 to p3trueid
dtypes: int64(101), object(21)
memory usage: 930.8+ MB


In [11]:
data.head()

,p0c1,p0c1isD,p0c1isH,p0c1isS,p0c2,p0c2isD,p0c2isH,p0c2isS,p0c3,p0c3isD,p0c3isH,p0c3isS,p0c4,p0c4isD,p0c4isH,p0c4isS,p0c5,p0c5isD,p0c5isH,p0c5isS,p1c1,p1c1isD,p1c1isH,p1c1isS,p1c2,p1c2isD,p1c2isH,p1c2isS,p1c3,p1c3isD,p1c3isH,p1c3isS,p1c4,p1c4isD,p1c4isH,p1c4isS,p1c5,p1c5isD,p1c5isH,p1c5isS,p2c1,p2c1isD,p2c1isH,p2c1isS,p2c2,p2c2isD,p2c2isH,p2c2isS,p2c3,p2c3isD,p2c3isH,p2c3isS,p2c4,p2c4isD,p2c4isH,p2c4isS,p2c5,p2c5isD,p2c5isH,p2c5isS,p3c1,p3c1isD,p3c1isH,p3c1isS,p3c2,p3c2isD,p3c2isH,p3c2isS,p3c3,p3c3isD,p3c3isH,p3c3isS,p3c4,p3c4isD,p3c4isH,p3c4isS,p3c5,p3c5isD,p3c5isH,p3c5isS,TC_power,TC_isD,TC_isH,TC_isS,trump_isD,trump_isH,trump_isS,caller,round,alone,winner1,winner2,winner3,winner4,winner5,points02,points13,played1,played2,played3,played4,played5,played6,played7,played8,played9,played10,played11,played12,played13,played14,played15,played16,played17,played18,played19,played20,result,p0trueid,p1trueid,p2trueid,p3trueid
0,20,0,1,0,12,0,1,0,10,0,0,0,1,0,0,0,2,0,0,1,35,0,1,0,31,1,0,0,4,0,0,0,3,0,0,0,1,0,0,1,10,1,0,0,4,1,0,0,1,1,0,0,5,0,0,1,4,0,0,1,30,0,1,0,5,1,0,0,2,1,0,0,10,0,0,1,3,0,0,1,15,0,1,0,0,1,0,0,1,0,0,0,3,0,3,1,0,JH0,9D1,AH2,9H3,JD0,QD1,TD2,TH3,QC0,QS1,JS2,AC3,9C3,JC0,KS1,KD2,9S0,AD1,AS2,QH3,Single,3,0,1,2
1,30,0,0,0,15,0,0,0,4,0,1,0,2,0,1,0,10,0,0,1,20,0,0,0,5,1,0,0,2,1,0,0,5,0,0,1,4,0,0,1,31,0,0,1,10,1,0,0,3,1,0,0,5,0,1,0,2,0,0,1,35,0,0,0,12,0,0,0,1,1,0,0,3,0,1,0,1,0,1,0,25,0,0,0,0,0,0,1,1,0,0,3,3,3,3,0,1,JC0,TC1,QC2,KC3,JH0,QH1,TD2,KH3,AD3,9D0,TH1,KD2,JS3,9C0,AC1,QS2,JD3,9H0,AS1,KS2,Single,0,1,2,3
2,3,0,0,0,5,0,1,0,10,0,0,1,5,0,0,1,1,0,0,1,30,1,0,0,25,1,0,0,20,1,0,0,12,1,0,0,2,0,1,0,31,0,1,0,10,0,0,0,2,0,0,0,4,0,1,0,2,0,0,1,35,1,0,0,4,0,0,0,1,0,0,0,1,0,1,0,3,0,0,1,15,1,0,0,1,0,0,0,1,0,2,0,0,0,0,2,0,AD0,JH1,JD2,TD3,QC2,JC3,9D0,TC1,KD0,TS1,9C2,KH3,QD0,QH1,9H2,KS3,TH0,AC1,JS2,AS3,Sweep,1,2,3,0
3,35,0,0,0,30,0,0,0,12,0,0,0,10,1,0,0,1,0,1,0,5,1,0,0,4,1,0,0,10,0,1,0,4,0,1,0,10,0,0,1,15,0,0,0,2,1,0,0,3,0,1,0,5,0,0,1,4,0,0,1,31,0,0,1,3,1,0,0,5,0,1,0,2,0,1,0,2,0,0,1,25,0,0,0,0,0,0,1,1,0,2,1,1,1,1,0,1,KH0,9H1,AH2,JH3,AS2,QS3,TS0,9C1,JC1,QD2,TC3,JS0,AD1,KD2,KS3,JD0,AC1,QH2,KC3,TH0,Single,2,3,0,1
4,10,0,0,0,4,0,0,0,10,1,0,0,2,0,1,0,1,0,1,0,5,1,0,0,4,1,0,0,1,1,0,0,5,0,1,0,4,0,1,0,31,0,0,0,25,0,0,1,2,0,0,0,1,0,0,0,3,0,1,0,30,0,0,1,20,0,0,1,12,0,0,1,5,0,0,0,2,1,0,0,35,0,0,1,0,0,1,1,1,0,1,2,3,3,2,0,1,KD0,KS1,TD2,AD3,JH1,9S2,TH3,QH0,KC2,AC3,9D0,9C1,JS3,QD0,JC1,QS2,QC3,KH0,TC1,AS2,Single,3,0,1,2


In [32]:
np.argwhere(np.isin(np.array([v[:-1] for v in data.iloc[0][['played'+str(i) for i in range(1, 21)]]]), 'KH')).reshape(-1)

array([], dtype=int64)

In [139]:
row = data.iloc[0]
current_player = 0
# input 0 gives actually player 3's hand
power_to_name = {1:'9', 2:'T', 3:'J', 4:'Q', 5:'K', 10:'A', 12:'9', 15:'T', 20:'Q', 25:'K', 30:'A', 31:'J', 35:'J'}
current_player_hand = [
                    power_to_name[row['p'+str(current_player)+'c'+str(ix)]] +\
                    'D'*row['p'+str(current_player)+'c'+str(ix)+'isD'] +\
                    'H'*row['p'+str(current_player)+'c'+str(ix)+'isH'] +\
                    'S'*row['p'+str(current_player)+'c'+str(ix)+'isS'] +\
                    'C'*(1-row['p'+str(current_player)+'c'+str(ix)+'isD']-\
                             row['p'+str(current_player)+'c'+str(ix)+'isH']-\
                             row['p'+str(current_player)+'c'+str(ix)+'isS'])\
                    for ix in range(1,6)]
print(current_player_hand)

['QH', '9H', 'AC', '9C', 'TS']


In [293]:
same_color_dict = {'C':'S', 'S':'C', 'H':'D', 'D':'H'}
names = ['9','T','J','Q','K','A']
suits = ['C', 'D', 'H', 'S']

trump_power = {'9':12, 'T':15, 'Q':20, 'K':25, 'A':30, 'left':31, 'right':35, None:0}
non_trump_power = {'9':1, 'T':2, 'J':3, 'Q':4, 'K':5, 'A':10, None:0}
power_to_name = {1:'9', 2:'T', 3:'J', 4:'Q', 5:'K', 10:'A', 12:'9', 15:'T', 20:'Q', 25:'K', 30:'A', 31:'J', 35:'J'}
COLS_PER_CARD = 14
def format_data_v1(data, usetqdm=True):
    # can milk 4x the training data for each trick
    # total of 20 training data points for each row
    formatted = np.zeros((len(data)*20, 24*14), dtype=np.uint8)
    correct_card = ['']*len(data)*20
    
    
    # each row is one game
    for i in notebook.tqdm(data.index) if usetqdm else data.index:
        row = data.iloc[i]
        trump_suit = 'D'*row['trump_isD'] + 'H'*row['trump_isH'] + 'S'*row['trump_isS']\
                        + 'C'*(1-row['trump_isD']-row['trump_isH']-row['trump_isS'])
        same_color_suit = same_color_dict[trump_suit]
        
        suits_and_suit_ixs = [trump_suit, same_color_suit]
        suits_and_suit_ixs += ('D' not in suits_and_suit_ixs)*['D', 'H'] + ('C' not in suits_and_suit_ixs)*['C', 'S']
        suits_and_suit_ixs = [[suits_and_suit_ixs[i], i] for i in range(len(suits_and_suit_ixs))]
        
        card_loc_and_owner = {}
        for card in [n+s for n in ['9','T','J','Q','K','A'] for s in ['C', 'D', 'H', 'S']]:
            spot = np.argwhere(np.array([v[:-1] for v in row[['played'+str(ix) for ix in range(1,21)]]])==card).reshape(-1)
            if len(spot) == 0: spot = [49]
            owner = (int(row['played'+str(spot[0]+1)][-1]))%4 if spot[0] != 49 else None
            card_loc_and_owner[card] = [int(spot[0])+1, owner]
        
        
        players_hands = {(current_player-1)%4:[
                    power_to_name[row['p'+str(current_player)+'c'+str(ix)]] +\
                    'D'*row['p'+str(current_player)+'c'+str(ix)+'isD'] +\
                    'H'*row['p'+str(current_player)+'c'+str(ix)+'isH'] +\
                    'S'*row['p'+str(current_player)+'c'+str(ix)+'isS'] +\
                    'C'*(1-row['p'+str(current_player)+'c'+str(ix)+'isD']-\
                             row['p'+str(current_player)+'c'+str(ix)+'isH']-\
                             row['p'+str(current_player)+'c'+str(ix)+'isS'])\
                    for ix in range(1,6)]
                for current_player in range(4)}
        
        for tricknum in range(5):
            for playernum in range(4):
                
                # [my, opp1, pner, opp2] x [hand, current trick, past trick] + [unknown, kitty]
                # [myhand, opp1hand, pnerhand, opp2hand, mycurrentrick, ...]
                current_spot = 4*tricknum + playernum
                current_player = int(row['played'+str(current_spot+1)][-1])
                
                correct_card[20*i + 4*tricknum + playernum] = row['played'+str(current_spot+1)][:-1]
                for suit, suit_ix in suits_and_suit_ixs:
                    for ix, name in enumerate(names):
                        formatted[20*i + 4*tricknum + playernum,\
                            suit_ix*(len(names)*COLS_PER_CARD) + ix*COLS_PER_CARD +\
                            spot_in_formatted(name, suit, card_loc_and_owner, current_player, \
                                              players_hands[current_player], current_spot, COLS_PER_CARD=COLS_PER_CARD)] = 1
                                              #current_player_hand, current_spot, COLS_PER_CARD=COLS_PER_CARD)] = 1
                
    return(formatted, correct_card)

In [291]:
def spot_in_formatted(name, suit, card_loc_and_owner, current_player, current_player_hand, current_spot, COLS_PER_CARD=14):

    power_to_name = {1:'9', 2:'T', 3:'J', 4:'Q', 5:'K', 10:'A', 12:'9', 15:'T', 20:'Q', 25:'K', 30:'A', 31:'J', 35:'J'}
    card_spot, card_owner = card_loc_and_owner[name+suit]
    card_owner = (card_owner-current_player)%4 if card_owner is not None else None

    # check if you're the dealer and you buried it
    if name+suit in current_player_hand and card_spot == 50:
        return COLS_PER_CARD-1

    # check if the card has already been played
    elif card_spot <= current_spot:
        # before this trick
        if card_spot <= (current_spot-(current_spot%4)):
            return 4*2 + card_owner
        # in this trick
        else:
            return 4 + card_owner

    # check if it's in your hand
    elif name+suit in current_player_hand:
        return 0

    # check if it's the turn card
    elif power_to_name[row['TC_power']] + 'D'*row['TC_isD'] + 'H'*row['TC_isH'] + 'S'*row['TC_isS'] +\
        'C'*(1-row['TC_isD']-row['TC_isH']-row['TC_isS']) == name+suit:
        # dealer has it
        if row['round'] == 1:
            return 3-current_player
        # it's in the kitty
        else:
            return COLS_PER_CARD-1

    # otherwise, just don't know
    else:
        return COLS_PER_CARD-2

In [249]:
p0hand = ['JH', 'JD', 'QC', 'JC', '9S']
p1hand = ['AD', 'QD', '9D', 'KS', 'QS']
p2hand = ['AH', 'KD', 'TD', 'AS', 'JS']
p3hand = ['QH', '9H', 'AC', '9C', 'TS'] + ['TH']

In [251]:
test, _ = format_data_v1(data.iloc[:1])

  0%|          | 0/1 [00:00<?, ?it/s]

In [294]:
formatted = format_data_v1(data)

  0%|          | 0/1000000 [00:00<?, ?it/s]